In [ ]:
import cv2 as cv
import os
import numpy as np
import pandas as pd
import random
import itertools
from scipy import signal

In [ ]:
finetune_test = 'test' # 'finetune', 'test'
data_path = '[USER_PATH]\\ArtificialVision\\data'
input_path = os.path.join(data_path, f'Middle_Resolution_177_unzipped_parcropped_128_removed_{finetune_test}')
folderPaths = [os.path.join(input_path, folder_name) for folder_name in os.listdir(input_path)]
output_path = os.path.join(data_path, f'Middle_Resolution_16_unzipped_parcropped_128_removed_{finetune_test}')
df = pd.read_csv(os.path.join(data_path, '210827_ANNA_Removing_uncontaminated_data.csv'))

In [ ]:
l = list(range(df.shape[0]))
l_2030_f = list(df.loc[(df['연령대'].isin(['20대','30대'])) & (df['성별']=='여')].index)
l_4050_f = list(df.loc[(df['연령대'].isin(['40대','50대'])) & (df['성별']=='여')].index)
l_2030_m = list(df.loc[(df['연령대'].isin(['20대','30대'])) & (df['성별']=='남')].index)
l_4050_m = list(df.loc[(df['연령대'].isin(['40대','50대'])) & (df['성별']=='남')].index)
l_20304050_f = list(df.loc[df['성별'] == '여'].index)
l_20304050_m = list(df.loc[df['성별'] == '남'].index)
n = 16

random.seed(22)
set_1 = random.sample(l, n)
sets = [set_1]

m = 0
input_folder = [df.iloc[i, 0] for i in sets[m]] 
assert len(input_folder) == 16
com_obj = itertools.combinations(input_folder, n)
com_list = list(com_obj)

com_list = com_list[0]
com_list = [i for i in com_list]
com_list = list(map(int, com_list))
com_list

In [ ]:
if finetune_test == 'finetune':
    camera_list = list(map(int, [4,5,6,7,8,9,10,14,15,16,17,18,19,20]))
    light_list = list(map(int, [1,2,3,4,5,6, 
                                8,9,10,11,12,13,14,15, 16,17,18,
                                19,20,21,22,23,24,25,26,27, 28,29,30])) 
    accessory_list = list(map(int, [1,2,3,5])) 
    expression_list = list(map(int, list(range(1, 4))))

    grayscale = [16]
    pixel = [128] 

elif finetune_test == 'test':
    camera_list = ['4','7','10'] 
    light_list = ['1']
    accessory_list =['1']
    expression_list = ['1','2','3'] 

    grayscale = [2,4,6,8,16]
    pixel = [16,24,32,64,128]

Make non-Gaussian-blurred (NGB) phosphene images

In [ ]:
for folderPath in folderPaths: 
    print(folderPath)

    folder_name = os.path.basename(folderPath) 
    os.makedirs(output_path+'\\'+folder_name, exist_ok=True) 
    person_output_path = output_path + '\\' + folder_name 
    filePaths = [os.path.join(folderPath, file_name) for file_name in os.listdir(folderPath)] 
        
    for filePath in filePaths: 
        file_name  = os.path.basename(filePath)
        
        sl_split = file_name.split('L')
        file_name2 = sl_split[-1]
        e_split = file_name2.split('E')
        file_name3 = e_split[-1]
        c_split = file_name3.split('C')
        file_name4 = c_split[-1]
        dot_split = file_name4.split('.')

        if (str(sl_split[0][-1]) in accessory_list) and (str(e_split[0]) in light_list) and (str(c_split[0][-1]) in expression_list) and (str(dot_split[0]) in camera_list):
            img = cv.imread(filePath, cv.IMREAD_GRAYSCALE) 

            # 1. Histogram equalization 
            hist_equalize_img = cv.equalizeHist(img) 

            # 2. Grayscaling
            for px in pixel:
                for gr in grayscale:  

                    gray_img = np.array(hist_equalize_img)
                    c = round(255/gr)

                    if gr == 2:
                        gray_img[(gray_img>=0) & (gray_img<c)] = 0 # black
                        gray_img[(gray_img>=c) & (gray_img<=255)] = 255 # white
                    
                    else:
                        for a in range(0, 256, c):
                            d = a+c
                            if d > 255: # exception
                                d = 255

                            if a == 0: # corresponding to the first bin
                                gray_img[(gray_img>=0) & (gray_img<d)] = 0
                            elif (255 - a) < c: # corresponding to the last bin
                                gray_img[(gray_img>=a) & (gray_img<d)] = 255
                            else:
                                gray_img[(gray_img>=a) & (gray_img<d)] = round((a+d)/2)

                    # 3. Pixelization
                    pixel_img = cv.resize(gray_img, dsize=(px,px), interpolation=cv.INTER_NEAREST)       
                    largest_pixel_size = max(pixel) # same output size
            
                    final_img = cv.resize(pixel_img, dsize=(128, 128), interpolation=cv.INTER_NEAREST) 

                    cv.imwrite(person_output_path
                        + '\\'
                        + '%dPIX_%dGS_'%(px, gr)
                        + os.path.basename(file_name),
                        final_img) 

Make Gaussian-blurred (GB) phosphene images

In [ ]:
for folderPath in folderPaths: 
    print(folderPath)

    folder_name = os.path.basename(folderPath) 
    os.makedirs(output_path+'\\'+folder_name, exist_ok=True) 
    person_output_path = output_path + '\\' + folder_name 
    filePaths = [os.path.join(folderPath, file_name) for file_name in os.listdir(folderPath)] 
        
    for filePath in filePaths: 
        file_name  = os.path.basename(filePath)
        
        sl_split = file_name.split('L')
        file_name2 = sl_split[-1]
        e_split = file_name2.split('E')
        file_name3 = e_split[-1]
        c_split = file_name3.split('C')
        file_name4 = c_split[-1]
        dot_split = file_name4.split('.')

        if (str(sl_split[0][-1]) in accessory_list) and (str(e_split[0]) in light_list) and (str(c_split[0][-1]) in expression_list) and (str(dot_split[0]) in camera_list):
            img = cv.imread(filePath, cv.IMREAD_GRAYSCALE)  

            # 1. Histogram equalization 
            hist_equalize_img = cv.equalizeHist(img) 

            # 2. Grayscaling
            for px in pixel:
                for gr in grayscale:  

                    gray_img = np.array(hist_equalize_img)
                    c = round(255/gr)

                    if gr == 2:
                        gray_img[(gray_img>=0) & (gray_img<c)] = 0 # black
                        gray_img[(gray_img>=c) & (gray_img<=255)] = 255 # white
                    
                    else:
                        for a in range(0, 256, c):
                            d = a+c
                            if d > 255: # exception
                                d = 255

                            if a == 0: # corresponding to the first bin
                                gray_img[(gray_img>=0) & (gray_img<d)] = 0
                            elif (255 - a) < c: # corresponding to the last bin
                                gray_img[(gray_img>=a) & (gray_img<d)] = 255
                            else:
                                gray_img[(gray_img>=a) & (gray_img<d)] = round((a+d)/2)

                    # 3. Pixelization
                    pixel_img = cv.resize(gray_img, dsize=(px,px), interpolation=cv.INTER_NEAREST)       
                    largest_pixel_size = max(pixel) # same output size
                   

                    # 4. Phosphenization  
                    k_size =  (largest_pixel_size // px) * 5
                    kernel1d = signal.gaussian(k_size, std=k_size/4).reshape(k_size, 1)
                    kernel2d = np.outer(kernel1d, kernel1d)
                    g_kernel_img = kernel2d

                    new_img = np.zeros((k_size*pixel_img.shape[0], k_size*pixel_img.shape[1]))
                    for i in range(0, pixel_img.shape[0]):
                        for j in range(0, pixel_img.shape[1]):
                            new_img[k_size*i:k_size*(i+1), k_size*j:k_size*(j+1)] = pixel_img[i, j] * g_kernel_img
                    
                    final_img = cv.resize(new_img, dsize=(128, 128), interpolation=cv.INTER_AREA)

                    cv.imwrite(person_output_path
                        + '\\'
                        + '%dPIX_%dGS_'%(px, gr)
                        + os.path.basename(file_name),
                        final_img) 